In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
import numpy as np

data = pd.read_csv('../cleaned_data_final.csv')

# Transforming the data so that each row contains one response and one label
responses = data['Human_response'].tolist() + data['ChatGPT_3_5_response'].tolist()
labels = [0] * len(data['Human_response']) + [1] * len(data['ChatGPT_3_5_response'])
data_combined = pd.DataFrame({'response': responses, 'label': labels})

# Text tokenization
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data_combined['response'].values)
X = tokenizer.texts_to_sequences(data_combined['response'].values)
X = pad_sequences(X)

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data_combined['label'])
y = to_categorical(integer_encoded)

class AttentionLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1),
                                 initializer="zeros")
        super(AttentionLayer, self).build(input_shape)
    def call(self, x):
        e = tf.nn.tanh(tf.add(tf.matmul(x, self.W), self.b))
        a = tf.nn.softmax(e, axis=1)
        output = x * a
        return tf.reduce_sum(output, axis=1)

# 5-fold cross-validation
n_folds = 5
kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
accuracies = []
y_labels = np.argmax(y, axis=1)

for train_index, test_index in kfold.split(X, y_labels):
    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    # Building the model
    model = tf.keras.Sequential([
        layers.Embedding(max_features, 128, input_length=X.shape[1]),
        layers.LSTM(64, return_sequences=True),
        AttentionLayer(),
        layers.Dense(2, activation='softmax')
    ])
    # Compiling the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train_fold, y_train_fold, epochs=5, batch_size=32, verbose=1)

    score, acc = model.evaluate(X_test_fold, y_test_fold, batch_size=32)
    accuracies.append(acc)

print("Mean Accuracy: %.2f" % (np.mean(accuracies)))

Epoch 1/5
50/50 [==============================] - 8s 97ms/step - loss: 0.6028 - accuracy: 0.7044